In [1]:
#Import our libraries
import time
import requests
import json
import xml.etree.ElementTree as ET  # for parsing XML
import xmltodict
import pandas as pd
import csv
import base64 
import pyodbc
import sqlalchemy as sal
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR

#Import our helper functions
import helper_functions_labor_distribution

# Step 1: Get the report file path that we need

In [2]:
report_path = helper_functions_labor_distribution.get_report_file_path('Employee List')
report_path

"/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/folder[@name='Export to SQL']/report[@name='Employee List']"

## Step 2: Retrieve the report

In [3]:
df = helper_functions_labor_distribution.retrieve_report(report_path) 
df 

,pay_group,ssn_(unformatted),employee_number,"employee_name_(last_suffix,_first_mi)",address_line_1_+_address_line_2,city,state,zip_code,email_address,alternate_email,...,org_level_3_code,termination_date,first_name,last_name,middle_name,supervisor_name_(first_mi_last_suffix),supervisor_employee_number,supervisor_email_address,location,location_code
0,Hourly BiWeekly,534118377,001003651,Aarestad_ Heidi H.,7320 Highland Dr,Everett,WA,98203,heidiaarestad@gmail.com,,...,000,2020-05-08 00:00:00,Heidi,Aarestad,H,Shelly K. Peltier,000250048,S.Peltier@proliancesurgeons.com,EDOC,002500
1,Hourly BiWeekly,534153592,001007204,Aasal_ Ashley E.,22500 10th Ave S,Des Moines,WA,98198,A.Aasal@proliancesurgeons.com,ashleydamon24@gmail.com,...,000,,Ashley,Aasal,E,Jessica L. Weathers,000260032,J.Weathers@proliancesurgeons.com,CSO - Payment Posting,000181
2,Hourly BiWeekly,607574894,001004881,Abad_ Queen Jacqueline D.,306 Kirkland Pl SE,Renton,WA,98056,,,...,000,2017-03-28 00:00:00,Queen Jacqueline,Abad,D,,,,PRET,001600
3,Salaried Biweekly,531965588,001008549,Abafo_ Johanna A.,18216 132nd Pl SE,Renton,WA,98058,j.abafo@proliancesurgeons.com,amoraj1434@gmail.com,...,000,,Johanna,Abafo,A,Erik M. Mortensen,001006786,E.Mortensen@proliancesurgeons.com,PRS - ASC,002601
4,Hourly BiWeekly,535045731,001001279,Abalahin_ Julie A.,10721 MT Tacoma Dr SW,Tacoma,WA,98498,j.abalahin@proliancesurgeons.com,,...,000,2021-06-04 00:00:00,Julie,Abalahin,A,,,,PSO - Lakewood ASC,007301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9581,Hourly BiWeekly,605583592,001004641,Zuniga_ Maritza,210 S 18th St,Mount Vernon,WA,98274,M.Zuniga@proliancesurgeons.com,,...,000,2019-05-03 00:00:00,Maritza,Zuniga,,,,,CENT,006700
9582,Hourly BiWeekly,179501094,001005696,Zweber_ Mary F.,8204 Beverly Ln,Everett,WA,98203,M.Zweber@proliancesurgeons.com,,...,000,2018-07-20 00:00:00,Mary,Zweber,F,,,,OSS - Ortho Clinic Tall,000200
9583,Hourly BiWeekly,534116910,001006118,Zwicky_ Amber D.,5209 NE 10th Pl,Renton,WA,98059,A.Zwicky@proliancesurgeons.com,,...,000,2019-05-31 00:00:00,Amber,Zwicky,D,Julia V. Stefoglo,001003126,J.Stefoglo@proliancesurgeons.com,PRET - ASC,001601
9584,Hourly BiWeekly,531374469,001006766,Zwolski_ Jane E.,12910 SE 240th ST Unit B4,Kent,WA,98031,J.Hackett@proliancesurgeons.com,janezwolski@gmail.com,...,000,,Jane,Zwolski,E,Sarah L. Peterson,001005009,SL.Peterson@proliancesurgeons.com,NWWH,004300


In [4]:
df.columns

Index(['pay_group', 'ssn_(unformatted)', 'employee_number',
       'employee_name_(last_suffix,_first_mi)',
       'address_line_1_+_address_line_2', 'city', 'state', 'zip_code',
       'email_address', 'alternate_email', 'birth_date', 'hourly_pay_rate',
       'annual', 'scheduled_work_hours', 'employment_type', 'org_level_1_code',
       'org_level_1', 'job_code', 'job', 'alternate_title', 'seniority_date',
       'last_hire_date', 'employment_status', 'org_level_2_code',
       'org_level_2', 'org_level_3_code', 'termination_date', 'first_name',
       'last_name', 'middle_name', 'supervisor_name_(first_mi_last_suffix)',
       'supervisor_employee_number', 'supervisor_email_address', 'location',
       'location_code'],
      dtype='object')

In [5]:
#Remove the null values so that they say 0 instead
df['hourly_pay_rate'] = df['hourly_pay_rate'].astype(str).replace("'@xs:nil': 'true'", '0')
df['annual'] = df['annual'].astype(str).replace("'@xs:nil': 'true'", '0')
df['org_level_1'] = df['org_level_1'].astype(str).replace("'@xs:nil': 'true'", '0')
df['job_code'] = df['job_code'].astype(str).replace("'@xs:nil': 'true'", '0')
df['org_level_2_code'] = df['org_level_2_code'].astype(str).replace("'@xs:nil': 'true'", '0')
df['org_level_3_code'] = df['org_level_3_code'].astype(str).replace("'@xs:nil': 'true'", '0')
df['supervisor_employee_number'] = df['supervisor_employee_number'].astype(str).replace("'@xs:nil': 'true'", '0')
df['location_code'] = df['location_code'].astype(str).replace("'@xs:nil': 'true'", '0')

In [6]:
df

,pay_group,ssn_(unformatted),employee_number,"employee_name_(last_suffix,_first_mi)",address_line_1_+_address_line_2,city,state,zip_code,email_address,alternate_email,...,org_level_3_code,termination_date,first_name,last_name,middle_name,supervisor_name_(first_mi_last_suffix),supervisor_employee_number,supervisor_email_address,location,location_code
0,Hourly BiWeekly,534118377,001003651,Aarestad_ Heidi H.,7320 Highland Dr,Everett,WA,98203,heidiaarestad@gmail.com,,...,000,2020-05-08 00:00:00,Heidi,Aarestad,H,Shelly K. Peltier,000250048,S.Peltier@proliancesurgeons.com,EDOC,002500
1,Hourly BiWeekly,534153592,001007204,Aasal_ Ashley E.,22500 10th Ave S,Des Moines,WA,98198,A.Aasal@proliancesurgeons.com,ashleydamon24@gmail.com,...,000,,Ashley,Aasal,E,Jessica L. Weathers,000260032,J.Weathers@proliancesurgeons.com,CSO - Payment Posting,000181
2,Hourly BiWeekly,607574894,001004881,Abad_ Queen Jacqueline D.,306 Kirkland Pl SE,Renton,WA,98056,,,...,000,2017-03-28 00:00:00,Queen Jacqueline,Abad,D,,,,PRET,001600
3,Salaried Biweekly,531965588,001008549,Abafo_ Johanna A.,18216 132nd Pl SE,Renton,WA,98058,j.abafo@proliancesurgeons.com,amoraj1434@gmail.com,...,000,,Johanna,Abafo,A,Erik M. Mortensen,001006786,E.Mortensen@proliancesurgeons.com,PRS - ASC,002601
4,Hourly BiWeekly,535045731,001001279,Abalahin_ Julie A.,10721 MT Tacoma Dr SW,Tacoma,WA,98498,j.abalahin@proliancesurgeons.com,,...,000,2021-06-04 00:00:00,Julie,Abalahin,A,,,,PSO - Lakewood ASC,007301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9581,Hourly BiWeekly,605583592,001004641,Zuniga_ Maritza,210 S 18th St,Mount Vernon,WA,98274,M.Zuniga@proliancesurgeons.com,,...,000,2019-05-03 00:00:00,Maritza,Zuniga,,,,,CENT,006700
9582,Hourly BiWeekly,179501094,001005696,Zweber_ Mary F.,8204 Beverly Ln,Everett,WA,98203,M.Zweber@proliancesurgeons.com,,...,000,2018-07-20 00:00:00,Mary,Zweber,F,,,,OSS - Ortho Clinic Tall,000200
9583,Hourly BiWeekly,534116910,001006118,Zwicky_ Amber D.,5209 NE 10th Pl,Renton,WA,98059,A.Zwicky@proliancesurgeons.com,,...,000,2019-05-31 00:00:00,Amber,Zwicky,D,Julia V. Stefoglo,001003126,J.Stefoglo@proliancesurgeons.com,PRET - ASC,001601
9584,Hourly BiWeekly,531374469,001006766,Zwolski_ Jane E.,12910 SE 240th ST Unit B4,Kent,WA,98031,J.Hackett@proliancesurgeons.com,janezwolski@gmail.com,...,000,,Jane,Zwolski,E,Sarah L. Peterson,001005009,SL.Peterson@proliancesurgeons.com,NWWH,004300


# Step 3: Export the report to a CSV file

In [7]:
df.to_csv('ukg_bulk_insert.csv', index=False)


# Step 4: Insert the report into BiData2 as a table

In [8]:
#helper_functions_labor_distribution.create_sql_table_v2('ZZ_TEST_Employee_List')

# Step 5: Close the API connection

In [9]:
helper_functions_labor_distribution.close_api_connection()

'<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/LogOffResponse</a:Action></s:Header><s:Body><LogOffResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><LogOffResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><ServiceId i:nil="true"/><ClientAccessKey>PLAB4</ClientAccessKey><Token i:nil="true"/><Status>LoggedOff</Status><StatusMessage i:nil="true"/><InstanceKey i:nil="true"/></LogOffResult></LogOffResponse></s:Body></s:Envelope>'

#Create table query
CREATE TABLE ZZ_TEST_Employee_List (
pay_group NVARCHAR(255),
 ssn NVARCHAR(255),
 employee_number NVARCHAR(255),
 employee_name NVARCHAR(255),
 address NVARCHAR(255),
 city NVARCHAR(255),
 state NVARCHAR(255),
 zip_code NVARCHAR(255),
 email_address NVARCHAR(255),
 alternate_email NVARCHAR(255),
 birth_date NVARCHAR(255),
 hourly_pay_rate NVARCHAR(255),
 annual NVARCHAR(255),
 scheduled_work_hours NVARCHAR(255),
 employment_type NVARCHAR(255),
 org_level_1_code NVARCHAR(255),
 org_level_1 NVARCHAR(255),
 job_code NVARCHAR(255),
 job NVARCHAR(255),
 alternate_title NVARCHAR(255),
 seniority_date NVARCHAR(255),
 last_hire_date NVARCHAR(255),
 employment_status NVARCHAR(255),
 org_level_2_code NVARCHAR(255),
 org_level_2 NVARCHAR(255),
 org_level_3_code NVARCHAR(255),
 termination_date NVARCHAR(255),
 first_name NVARCHAR(255),
 last_name NVARCHAR(255),
 middle_name NVARCHAR(255),
 supervisor_name NVARCHAR(255),
 supervisor_employee_number NVARCHAR(255),
 supervisor_email_address NVARCHAR(255),
 location NVARCHAR(255),
 location_code NVARCHAR(255))

